Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

# LAB9

Write a local-search algorithm (eg. an EA) able to solve the *Problem* instances 1, 2, 5, and 10 on a 1000-loci genomes, using a minimum number of fitness calls. That's all.

### Deadlines:

* Submission: Sunday, December 3 ([CET](https://www.timeanddate.com/time/zones/cet))
* Reviews: Sunday, December 10 ([CET](https://www.timeanddate.com/time/zones/cet))

Notes:

* Reviews will be assigned  on Monday, December 4
* You need to commit in order to be selected as a reviewer (ie. better to commit an empty work than not to commit)

### Conclusions:

Hello fellow reviewer, welcome in my version of lab9. <br>
Just a small heads up: I failed <br>
I managed to solve only problem 1 and 2, even though I spent a lot of time i didn't manage to solve problem 5 and 10 <br>
Here you can learn about my versions of extintion and islands, I implemented also other things, but couldn't make those work <br>
Hope you will enjoy reviewing my code :D

### Teammates
* Worked with Massimo Porcheddu and Miriam Ivaldi


In [61]:
from random import random, choice, randint
from functools import reduce
from collections import namedtuple
from dataclasses import dataclass
from copy import deepcopy
import functools

from pprint import pprint

import numpy as np

import lab9_lib

In [54]:
fitness = lab9_lib.make_problem(2)




In [55]:
NUM_GENERATIONS = 10000
NUM_SETS = 1000
NUM_MIGRATIONS = 10


In [56]:
#every kind of mutation/crossoevr i've tried is here. Most of them were useless since they didn't improve any result

@dataclass
class Individual:
    fitness: tuple
    genotype: list[bool]


def select_female(pop):
    females = [element for element in pop if element.genotype.count(True)  & 1 == 0]
    if(len(females) == 0):
        return Individual(
            genotype = [0] + [choice((0, 1)) for _ in range(NUM_SETS - 1)],
            fitness=None,
        )
    pool = [choice(females) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def select_male(pop):
    males = [element for element in pop if element.genotype.count(True)  & 1 == 1]
    if(len(males) == 0):
        return Individual(
            genotype = [1] + [choice((0, 1)) for _ in range(NUM_SETS - 1)],
            fitness=None,
        )
    pool = [choice(males) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion


def select_parent(pop):
    pool = [choice(pop) for _ in range(TOURNAMENT_SIZE)]
    champion = max(pool, key=lambda i: i.fitness)
    return champion

def mutate(ind: Individual) -> Individual:
    offspring = deepcopy(ind)
    pos = randint(0, NUM_SETS-1)
    offspring.genotype[pos] = 1 - offspring.genotype[pos]
    offspring.fitness = None
    return offspring

def mutate_random(ind: Individual) -> Individual:
    offspring = deepcopy(ind)
    for index, i in enumerate(offspring.genotype):
        if(random()< 0.3):
            offspring.genotype[index] = 1 - offspring.genotype[index]
    offspring.fitness = None
    return offspring

def logical_right_shift(ind: Individual) -> Individual:
    # Convert boolean array to integer representation
    offspring = deepcopy(ind)

    int_value = sum(bit << i for i, bit in enumerate(ind.genotype))
    
    # Perform logical right shift
    shifted_value = int_value >> randint(1,5)
    
    # Convert back to boolean array
    shifted_bool_array = [(shifted_value & (1 << i)) != 0 for i in range(len(ind.genotype))]
    offspring.genotype= shifted_bool_array
    offspring.fitness = None
    
    return offspring

def logical_right_shift_and_mutate(ind: Individual) -> Individual:
    # Convert boolean array to integer representation
    offspring = deepcopy(ind)

    int_value = sum(bit << i for i, bit in enumerate(ind.genotype))
    
    # Perform logical right shift
    shifted_value = int_value >> randint(1,10)
    
    # Convert back to boolean array
    shifted_bool_array = [(shifted_value & (1 << i)) != 0 for i in range(len(ind.genotype))]
    offspring.genotype= shifted_bool_array
    pos = randint(0, NUM_SETS-1)
    offspring.genotype[pos] = 1 - offspring.genotype[pos]
    offspring.fitness = None
    
    return offspring

def logical_left_shift_and_mutate(ind: Individual) -> Individual:
    # Convert boolean array to integer representation
    offspring = deepcopy(ind)

    int_value = sum(bit >> i for i, bit in enumerate(ind.genotype))
    
    # Perform logical right shift
    shifted_value = int_value >> randint(1,10)
    
    # Convert back to boolean array
    shifted_bool_array = [(shifted_value & (1 >> i)) != 0 for i in range(len(ind.genotype))]
    offspring.genotype= shifted_bool_array
    pos = randint(0, NUM_SETS-1)
    offspring.genotype[pos] = 1 - offspring.genotype[pos]
    offspring.fitness = None
    
    return offspring

def logical_left_shift(ind: Individual) -> Individual:
    # Convert boolean array to integer representation
    offspring = deepcopy(ind)

    int_value = sum(bit >> i for i, bit in enumerate(ind.genotype))
    
    # Perform logical right shift
    shifted_value = int_value >> randint(0,1000)
    
    # Convert back to boolean array
    shifted_bool_array = [(shifted_value & (1 >> i)) != 0 for i in range(len(ind.genotype))]
    offspring.genotype= shifted_bool_array
    offspring.fitness = None
    
    return offspring

# Yup i know this is cheating, i was just wondering if i could manage to finish problem 5 even cheating and i failed. Imagine losing even when cheating :(

def mutate1(ind: Individual) -> Individual:
    offspring = deepcopy(ind)
    true_indices = [i for i, value in enumerate(offspring.genotype) if value == 0]
    if(len(true_indices)==0):
        return ind
    pos = choice(true_indices)

    offspring.genotype[pos] = 1 
    offspring.fitness = None
    return offspring

def mutate0(ind: Individual) -> Individual:
    offspring = deepcopy(ind)
    true_indices = [i for i, value in enumerate(offspring.genotype) if value == 1]
    pos = choice(true_indices)

    offspring.genotype[pos] = 0
    offspring.fitness = None
    return offspring


def inversion_mutation(ind: Individual) -> Individual:
    offspring =     Individual(
        genotype = ind.genotype[::-1],
        fitness = None,
    )
    return offspring

def insert_mutation(ind: Individual) -> Individual:
    offspring = deepcopy(ind)

    # Get two casual position the first coud be at least 3 position before the end
    pos_1 = randint(0, len(ind.genotype)-3)
    pos_2 = randint(pos_1+1, len(ind.genotype)-1)

    #do the insert
    offspring.genotype[pos_1+1] = ind.genotype[pos_2]
    for i in range(pos_1+2, pos_2+1):
        offspring.genotype[i] = ind.genotype[i-1]
        
    return offspring

# This was also cheating, i did a 100_cut version of this and i could finish problem 10, however duplication sounds cheating to me 

def ten_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, 99)
    array_length = 1000
    num_slices = 10

    # Calculate the size of each slice
    slice_size = array_length // num_slices

    # Create 10 slices
    seed1 = [ind1.genotype[i:i+slice_size] for i in range(cut_point, array_length-100, slice_size)]
    seed2 = [ind2.genotype[i:i+slice_size] for i in range(cut_point, array_length-100, slice_size)]
    champion = max(seed1 + seed2, key=lambda i: fitness(i))
    champion[cut_point] = 1 - champion[cut_point]

    offspring = Individual(fitness=None, genotype=champion * 10)
    assert len(offspring.genotype) == NUM_SETS
    return offspring


def one_cut_xover(ind1: Individual, ind2: Individual) -> Individual:
    cut_point = randint(0, NUM_SETS-1)
    offspring = Individual(fitness=None,
                           genotype=ind1.genotype[:cut_point] + ind2.genotype[cut_point:])
    assert len(offspring.genotype) == NUM_SETS
    return offspring

def xor_xover(ind1: Individual, ind2: Individual) -> Individual:
    
    offspring = Individual(fitness=None,
                           genotype= [a ^ b for a, b in zip(ind1.genotype, ind2.genotype)]
)
    assert len(offspring.genotype) == NUM_SETS
    return offspring

# This was clearly a cheat as well, the lack of diversity brings the result an array full of 1
# I also implemented or_crossover which was clearly cheating as well

def not_xor_xover(ind1: Individual, ind2: Individual) -> Individual:
    offspring = Individual(fitness=None,
                           genotype= [(1 - (a ^ b)) for a, b in zip(ind1.genotype, ind2.genotype)]
)
    assert len(offspring.genotype) == NUM_SETS
    return offspring

def and_xover(ind1: Individual, ind2: Individual) -> Individual:
    offspring = Individual(fitness=None,
                           genotype= [a and b for a, b in zip(ind1.genotype, ind2.genotype)]
)
    assert len(offspring.genotype) == NUM_SETS
    return offspring

def two_cut_xover(ind1: Individual, ind2: Individual) -> (Individual):
    #check that I have two genotypes made up of at least 3 elements
    assert len(ind1.genotype) > 2 and len(ind2.genotype) > 2
    
    #fisrt cut can't be the first element and neither the 2 before the end (end included)
    first_cut = randint(1, len(ind2.genotype)-2)
    #second cut can't be the second and neither the 1 before the end (end included)
    second_cut = randint(first_cut+1, len(ind2.genotype)-1)
    #print(first_cut, second_cut)

    #create the new child with the new cut
    offspring_1 = Individual(fitness=None,
                           genotype=ind1.genotype[:first_cut] + ind2.genotype[first_cut:second_cut] + ind1.genotype[second_cut:])
    #create the new child with the new cut
    offspring_2 = Individual(fitness=None,
                           genotype=ind2.genotype[:first_cut] + ind1.genotype[first_cut:second_cut] + ind2.genotype[second_cut:])

    assert (len(offspring_1.genotype) == NUM_SETS) and (len(offspring_2.genotype) == NUM_SETS)

    return offspring_1

def uniform_xover(ind1: Individual, ind2: Individual) -> (Individual):
    #check that I have two genotypes of the same lenght
    assert len(ind1.genotype) == len(ind2.genotype) 
    
    #create the new child with the new cut
    offspring_1 = Individual(fitness=None, genotype=[])
    #create the new child with the new cut
    offspring_2 = Individual(fitness=None, genotype=[])

    #fill the two of their genotype
    for i in range(NUM_SETS):
        if(choice((0, 1)) == 1):
            offspring_1.genotype.append(ind1.genotype[i])
            offspring_2.genotype.append(ind2.genotype[i])
        else:
            offspring_1.genotype.append(ind2.genotype[i])
            offspring_2.genotype.append(ind1.genotype[i])

    assert (len(offspring_1.genotype) == NUM_SETS) and (len(offspring_2.genotype) == NUM_SETS)
    return offspring_1


In [57]:
def gen_pop(population_size):
    population = [
        Individual(
            genotype=[choice((0,1)) for _ in range(NUM_SETS)],
            fitness=None,
        )
        for _ in range(population_size)
    ]

    for i in population:
        i.fitness = fitness(i.genotype)
    return population

In [58]:
# function to remove duplicated
def remove_duplicates(sorted_list):
    # Check if the list is empty or has only one element
    if len(sorted_list) < 2:
        return sorted_list

    # Initialize a new list with the first element from the sorted list
    result = [sorted_list[0]]

    # Iterate through the sorted list, and only add elements that are different from the previous one
    for i in range(1, len(sorted_list)):
        if sorted_list[i] != sorted_list[i - 1]:
            result.append(sorted_list[i])

    return result

# Population goes BOOM! (if you lack humor sense and are too sensitive on this topic, just imagine a meteor :D )

def nuke(nuc, timer, population):
    if(timer == 0):
        nuc[timer] = population[0].fitness
        timer += 1

    else:
        if(population[0].fitness == nuc[timer-1] ):

            nuc[timer] = population[0].fitness
            timer += 1
            if(timer >=5):
                population = population[:POPULATION_SIZE//10]
                population.extend(gen_pop(9* (POPULATION_SIZE//10)))
                timer = 0
                print("BOOM!")
        else:
            timer = 1
            nuc[0] = population[0].fitness
    return nuc, timer, population





In [59]:
POPULATION_SIZE = 100 #Hope these guys are ready for some cool darwinism
OFFSPRING_SIZE = 200
TOURNAMENT_SIZE = 2
MUTATION_PROBABILITY = .3

population = gen_pop(POPULATION_SIZE)
island = gen_pop(POPULATION_SIZE)
timer = 0
timer_island = 0
nuc_island = list(range(10))
nuc = list(range(10))

In [60]:
for generation in range(NUM_GENERATIONS):


    #Migration is done here
    if(generation % (NUM_GENERATIONS//NUM_MIGRATIONS) == 0):
        print(sum(island[0].genotype))
        island.extend(population[:POPULATION_SIZE//20])
    elif(generation % (NUM_GENERATIONS//NUM_MIGRATIONS) == NUM_GENERATIONS//(2 *NUM_MIGRATIONS)):
        print(sum(population[0].genotype))
        population.extend(island[:POPULATION_SIZE//20])
        print("migrazione!!! -----------------------------------------")
    
    offspring = list()
    offspring_island = list()

    for counter in range(OFFSPRING_SIZE):
        if random() < MUTATION_PROBABILITY:  # self-adapt mutation probability
            # mutation  copy pasted for laziness, island model was difficilt to balance, so i didn't try with more than 2 island.
            # Would've implemented something better if i managed to solve problem 5, unluckily this is not the case, so i just gave up
            p = select_parent(population)
            o = logical_right_shift_and_mutate(p)
            pi = select_parent(island)
            oi = mutate(pi)
        else:
            # xover # add more xovers
            p1 = select_parent(population)
            p2 = select_parent(population)
            o = two_cut_xover(p1,p2)

            pi1 = select_parent(island)
            pi2 = select_parent(island)
            oi = one_cut_xover(pi1,pi2)


        offspring.append(o)
        offspring_island.append(oi)

    for i in range(OFFSPRING_SIZE):
        offspring[i].fitness = fitness(offspring[i].genotype)
        offspring_island[i].fitness = fitness(offspring_island[i].genotype)

    # More code redundancy :D
    
    population.extend(offspring)
    island.extend(offspring_island)
    population.sort(key=lambda i: i.fitness, reverse=True)
    island.sort(key=lambda i: i.fitness, reverse=True)

    # Remove duplicate and apply extintion pattern which is the first one i developed and the only one that got me somewhere (i finished problem 2 even without islands)

    population = remove_duplicates(population)
    island = remove_duplicates(island)
    population = population[:POPULATION_SIZE]
    island = island[:POPULATION_SIZE]
    print(population[0].fitness, fitness.calls, generation, island[0].fitness)

    #when bomb expldes, 90% of population is deleted

    nuc, timer, population = nuke(nuc, timer, population)
    nuc_island, timer_island, island = nuke(nuc_island, timer_island, island)

    # Warning debug meaning the 2 populations have same top individual (lack of diversity)
    if(population[0].genotype == island[0].genotype):
        print("----------------------------------")

    # Stop when fitness reaches 1
    if(population[0].fitness == 1 or island[0].fitness == 1):
        break

print(population[0].genotype,island[0].fitness, fitness(population[0].genotype))
print(fitness.calls)



491
0.524 600 0 0.516
0.524 1000 1 0.524
0.524 1400 2 0.53
0.534 1800 3 0.53
0.534 2200 4 0.546
0.534 2600 5 0.546
0.534 3000 6 0.56
0.534 3400 7 0.56
BOOM!
0.536 3890 8 0.568
0.536 4290 9 0.568
0.55 4690 10 0.568
0.55 5090 11 0.568
0.55 5490 12 0.568
BOOM!
0.55 5980 13 0.568
0.55 6380 14 0.568
BOOM!
0.55 6870 15 0.57
0.55 7270 16 0.57
0.55 7670 17 0.57
0.552 8070 18 0.57
0.552 8470 19 0.57
BOOM!
0.554 8960 20 0.57
0.554 9360 21 0.57
0.554 9760 22 0.572
0.556 10160 23 0.572
0.556 10560 24 0.572
0.556 10960 25 0.572
0.556 11360 26 0.572
BOOM!
0.556 11850 27 0.572
BOOM!
0.558 12340 28 0.572
0.558 12740 29 0.572
0.56 13140 30 0.574
0.56 13540 31 0.574
0.562 13940 32 0.574
0.562 14340 33 0.574
0.566 14740 34 0.576
0.566 15140 35 0.576
0.57 15540 36 0.576
0.57 15940 37 0.576
0.57 16340 38 0.576
BOOM!
0.572 16830 39 0.576
0.574 17230 40 0.578
0.574 17630 41 0.58
0.574 18030 42 0.58
0.576 18430 43 0.58
0.576 18830 44 0.58
0.578 19230 45 0.58
BOOM!
0.58 19720 46 0.58
0.58 20120 47 0.582
0.584 